# The Diet Problem

In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
import os

Have a look at the diet problem data:

In [ ]:
thepath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]
filename = 'data_mec_optim\\lp_stigler-diet\\StiglerData1939.txt'
thedata = pd.read_csv(thepath + filename, sep='\t')
thedata = thedata.dropna(how = 'all')
thedata

Isolate model inputs:

In [ ]:
commodity = thedata['Commodity'].values
intake = thedata.iloc[:, 4:].fillna(0).values
allowance = thedata.iloc[-1, 4:].fillna(0).values

In [ ]:
commodity = (thedata['Commodity'].values)[:-1]
intake = thedata.iloc[:-1, 4:].fillna(0).transpose().values
allowance = thedata.iloc[-1, 4:].fillna(0).transpose()

Set up the model: note that Gurobi in Python is different from R, in the sense that we cannot a matrix of constraints

In [ ]:
m = grb.Model('optimalDiet')
meal = m.addVars(commodity, name='meal')
m.setObjective(meal.sum(), grb.GRB.MINIMIZE)
m.addConstrs((grb.quicksum(meal[k] * intake[i, j] for j, k in enumerate(commodity)) >= allowance[i]
              for i in range(intake.shape[0])), name='c')

In [ ]:
m.optimize()
if m.status == grb.GRB.Status.OPTIMAL:
    total = 0
    solution = m.getAttr('x', meal)
    pi = m.getAttr('pi')

In [ ]:
print('***Optimal solution***')
for food in commodity:
    if solution[food] > 0:
        print(food, solution[food] * 365)
        total += solution[food] * 365
print('Total cost (optimal) =', total)

As promised, we achieve the minimum cost bundle at $\$39.67$ per year in $1939$ dollars. If we compare this to Stigler's solutions which was 

|Food| Annual Quantities| Annual Cost|
| ---------- | ------------------ | ------------ |
| Wheat Flour | 	370 lb.|   \$13.33 |
| Evaporated Milk | 	57 cans |	  \$3.84 |
|Cabbage| 	111 lb. 	  |\$4.11|
|Spinach| 	23 lb. 	  |\$1.85|
|Dried Navy Beans| 	285 lb. |	\$16.80|
|Total Annual Cost| 	&nbsp; 	| \$39.93 |